In [1]:
### Loading the required libraries 
import numpy as np
from projection_simplex import projection_simplex_bisection as proj ### loading the simplex projection module

In [2]:
class COCO:
    def __init__(self, n, T): ### n denotes the dimension of input
        self.D=np.sqrt(2) ### D denotes the Euclidean diameter of the feasible set, which is sqrt(2) for simplex
        self.n=n 
        self.x=np.array(proj(np.random.rand(n))).reshape(-1,1) ### n denotes the input dimension
        self.step_size=0
        self.grad_sum_sq=0 ### internal variable needed to compute the step size
        self.Q=0 ### initializing the cumulative constraint violation
        self.TotalCost=0 ### initializing the total cost incurred
        self.Lambda=1.0/(2*np.sqrt(T)) 
        self.V=1   ## setting the required parameters
    
        
        
        
    def predict_COCO(self, grad):
        self.x=np.array(proj(self.x-self.step_size*grad)).reshape(-1,1)
        return
    
    def update_COCO(self, cost_val, constr_val):
        self.Q = self.Q+constr_val
        self.TotalCost=self.TotalCost+cost_val ## updating the total violation and cost
    
    def surrogate_cost_grad(self, cost_grad, constr_grad): # returns the gradient of the surrogate cost function at x
        grad=self.V*cost_grad.reshape(-1,1)+self.Lambda*np.exp(self.Lambda*self.Q)*constr_grad.reshape(-1,1)
        self.grad_sum_sq=self.grad_sum_sq+np.linalg.norm(grad)**2
        self.step_size=self.D/np.sqrt(2.0*self.grad_sum_sq) ### selecting step sizes using AdaGrad
        return grad.reshape(-1,1)
        
        
    
        

In [3]:
class Adversary_2Player:
    
    def __init__(self, n): # we are considering the single constraint case, i.e., m=1
        self.n=n
        self.A=np.random.randn(n,n)
        self.C_x=np.random.rand(n,1)
        self.C_y=np.random.rand(n,1)
    
    def grad_f(self, x):
        best_action=np.argmax((self.A.T)@x) # computes the current best action of the adversary
        return self.A[:,best_action].reshape(-1,1) # returns a column vector
    
    def f_val(self, x):
        best_action=np.argmax((self.A.T)@x) # computes the current best action of the adversary
        return (x.T@self.A[:,best_action])
    
    def grad_g(self, x):
        best_action=np.argmax((self.A.T)@x) # computes the current best action of the adversary
        if self.C_x.T@x+self.C_y[best_action]-1>0:
            return self.C_x.reshape(-1,1)
        else:
            return np.zeros([int(self.n),1])
    
    def g_val(self,x):
        best_action=np.argmax((self.A.T)@x) # computes the current best action of the adversary
        return(max(0,self.C_x.T@x+self.C_y[best_action]-1))

In [4]:
class Adversary_Test:
### solving the following problem
### https://www.wolframalpha.com/input?i=minimize+e%5Ex%2By%5E2%2C+s.t.+x%2By%3D1%2C+x%3E%3D0%2C+y%3E%3D0%2C+x%5E2%2B2.5*y%5E2%3C%3D1
    def __init__(self,n):
        self.n=n
        
    def grad_f(self, x):
        return np.array([np.exp(x[0]), 2*x[1]]).reshape(-1,1)
    
    def grad_g(self, x):
        if x[0]**2+2.5*x[1]**2-1>=0:
            return np.array([2.0*x[0], 5*x[1]]).reshape(-1,1)
        else:
            return np.zeros([self.n,1])
                             
    def f_val(self,x):
        return (np.exp(x[0])+x[1]**2)
                             
    def g_val(self, x):
        return max(0, x[0]**2+2.5*x[1]**2-1)
        

In [5]:
### Implements the driver code

T=10000 # number of steps -- seems like T needs to be large for the constraint to be satisfied if the cost function
## is not strongly convex
n=2   # dimension of the action

coco = COCO(n,T) ### Algorithm object
adversary= Adversary_Test(n) ### Adversary object

for t in range(T):
    current_action=coco.x # getting the current action of the algorithm

    cost_grad=adversary.grad_f(current_action)
    constr_grad=adversary.grad_g(current_action)
    cost_val=adversary.f_val(current_action)
    constr_val=adversary.g_val(current_action)  ### getting adversary's choices
    
    coco.update_COCO(cost_val, constr_val) ### updating internal states of the algorithm
    surrogate_cost_grad=coco.surrogate_cost_grad(cost_grad, constr_grad) ### computing the gradient of the surrogate cost
    
    coco.predict_COCO(surrogate_cost_grad)  ### predicting the next_action
    
   


In [6]:
sum(coco.x)

array([0.99997624])

In [7]:
coco.x

array([[0.42839536],
       [0.57158088]])